In [1]:
import polars as pl
import re

from collections import defaultdict
from itertools import combinations

# part1

In [2]:
day16_inputs = "inputs/day16.txt"
with open(day16_inputs, 'r') as file:
    day16_data = file.readlines()

In [3]:
sue_template = {
    "sue": None,
    "children": None,
    "cats": None,
    "samoyeds": None,
    "pomeranians": None,
    "akitas": None,
    "vizslas": None,
    "goldfish": None,
    "trees": None,
    "cars": None,
    "perfumes": None
}

schema = {col: int for col in sue_template.keys()}
sues_df = pl.DataFrame(schema=schema)

In [4]:
for line in day16_data:
    line_split = re.match(r"^(Sue \d+): (.*)$", line.strip())

    sue_info = line_split.group(1)
    mfcsam_info = line_split.group(2)
    
    sue_num = int(re.findall(r'\d+', sue_info)[0])
    
    mfcsam_info_split = mfcsam_info.split(", ")
    new_row = {**sue_template, "sue": sue_num}
    for item in mfcsam_info_split:
        item_split = item.split(": ")
        mfcsam_attr = item_split[0]
        mfcsam_value = int(item_split[1])
        new_row[mfcsam_attr] = mfcsam_value
        
    new_row_df = pl.DataFrame([new_row])
    sues_df = pl.concat([sues_df, new_row_df])

In [5]:
mfcsam_sue = {
    "children": 3,
    "cats": 7,
    "samoyeds": 2,
    "pomeranians": 3,
    "akitas": 0,
    "vizslas": 0,
    "goldfish": 5,
    "trees": 3,
    "cars": 2,
    "perfumes": 1
}
mfcsam_keys = list(mfcsam_sue.keys())

In [6]:
print(sues_df)

shape: (500, 11)
┌─────┬──────────┬──────┬──────────┬───┬──────────┬───────┬──────┬──────────┐
│ sue ┆ children ┆ cats ┆ samoyeds ┆ … ┆ goldfish ┆ trees ┆ cars ┆ perfumes │
│ --- ┆ ---      ┆ ---  ┆ ---      ┆   ┆ ---      ┆ ---   ┆ ---  ┆ ---      │
│ i64 ┆ i64      ┆ i64  ┆ i64      ┆   ┆ i64      ┆ i64   ┆ i64  ┆ i64      │
╞═════╪══════════╪══════╪══════════╪═══╪══════════╪═══════╪══════╪══════════╡
│ 1   ┆ null     ┆ null ┆ null     ┆ … ┆ 0        ┆ null  ┆ 9    ┆ null     │
│ 2   ┆ 3        ┆ null ┆ 9        ┆ … ┆ null     ┆ null  ┆ null ┆ null     │
│ 3   ┆ 4        ┆ null ┆ null     ┆ … ┆ null     ┆ 6     ┆ 6    ┆ null     │
│ 4   ┆ null     ┆ null ┆ null     ┆ … ┆ 9        ┆ 4     ┆ null ┆ null     │
│ 5   ┆ null     ┆ null ┆ null     ┆ … ┆ null     ┆ null  ┆ 5    ┆ null     │
│ …   ┆ …        ┆ …    ┆ …        ┆ … ┆ …        ┆ …     ┆ …    ┆ …        │
│ 496 ┆ null     ┆ null ┆ null     ┆ … ┆ null     ┆ 5     ┆ null ┆ null     │
│ 497 ┆ null     ┆ null ┆ null     ┆ … ┆ null  

In [15]:
mfcsam_search_combos = [comb for i in range(3, 4) for comb in combinations(mfcsam_keys, i)]

sues_matching_combos = defaultdict(int)
for combo in mfcsam_search_combos:
    mask = (pl.col(combo[0]) == mfcsam_sue[combo[0]]) & \
       (pl.col(combo[1]) == mfcsam_sue[combo[1]]) & \
       (pl.col(combo[2]) == mfcsam_sue[combo[2]])
    filtered_df = sues_df.filter(mask)
    if filtered_df.height > 0:
        for idx, row in enumerate(filtered_df.iter_rows(named=True)):
            sue_num = row["sue"]
            matched_values = {key: row[key] for key in combo}
            print(f"Sue {sue_num} matched values: {matched_values}")
            sues_matching_combos[sue_num] += 1

print(sues_matching_combos)
correct_sue = max(sues_matching_combos, key=sues_matching_combos.get)
print(f"Correct Sue: {correct_sue}")

Sue 373 matched values: {'pomeranians': 3, 'vizslas': 0, 'perfumes': 1}
defaultdict(<class 'int'>, {373: 1})
Correct Sue: 373


# part2

In [17]:
sues_matching_combos = defaultdict(int)
for combo in mfcsam_search_combos:
    mask_parts = []
    for key in combo:
        if key == "cats" or key == "trees":
            mask_parts.append(pl.col(key) > mfcsam_sue[key])
        elif key == "pomeranians" or key == "goldfish":
            mask_parts.append(pl.col(key) < mfcsam_sue[key])
        else:
            mask_parts.append(pl.col(key) == mfcsam_sue[key])
    
    mask = mask_parts[0]
    for part in mask_parts[1:]: 
        mask &= part
    
    filtered_df = sues_df. filter(mask)
    if filtered_df.height > 0:
        for idx, row in enumerate(filtered_df.iter_rows(named=True)):
            sue_num = row["sue"]
            matched_values = {key: row[key] for key in combo}
            print(f"Sue {sue_num} matched values: {matched_values}")
            sues_matching_combos[sue_num] += 1

print(sues_matching_combos)
correct_sue = max(sues_matching_combos, key=sues_matching_combos.get)
print(f"Correct Sue: {correct_sue}")

Sue 260 matched values: {'samoyeds': 2, 'vizslas': 0, 'goldfish': 0}
defaultdict(<class 'int'>, {260: 1})
Correct Sue: 260
